<a href="https://colab.research.google.com/github/Atonbom/KimPyRaptor/blob/main/Python/Tomato_Weight_Estimation_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Datasets

**Download datasets**

In [ ]:
#Merlice dataset
#I build this dataset myself
!gdown --id 1AQe2AyMREwxhtXEm24isNsKR7fjKkFs2
!unzip '/content/tomato_dataframes_merlice.zip'
!rm '/content/tomato_dataframes_merlice.zip'

**Define dataframe for training**

In [ ]:
#tomato_dataframe = '/content/tomato_dataframe_tros.csv'
#tomato_dataframe = '/content/tomato_dataframe_tros_5x.csv'
#tomato_dataframe = '/content/tomato_dataframe_cocktail_tros.csv'
#tomato_dataframe = '/content/tomato_dataframe_cocktail_tros_5x.csv'
#tomato_dataframe = '/content/tomato_dataframe_cherry.csv'
#tomato_dataframe = '/content/tomato_dataframe_cherry_5x.csv'
#tomato_dataframe = '/content/tomato_dataframe_all_datasets_single.csv'
#tomato_dataframe = '/content/tomato_dataframe_all_datasets_5x.csv'
#tomato_dataframe = '/content/tomato_dataframe_all_datasets.csv'
#tomato_dataframe = '/content/tomato_dataframe_merlice_batch01_20d.csv'
#tomato_dataframe = '/content/tomato_dataframe1x20d01_30.csv'
#tomato_dataframe = '/content/tomato_dataframe_merlice_all.csv'
tomato_dataframe = '/content/tomato_dataframe_merlice_all_but01.csv'
#tomato_dataframe = '/content/tomato_dataframe_merlice_all_but02.csv'
#tomato_dataframe = '/content/tomato_dataframe_merlice_all_but03.csv'
#tomato_dataframe = '/content/tomato_dataframe_merlice_all_but04.csv'
#tomato_dataframe = '/content/tomato_dataframe_merlice_all_but05.csv'
#tomato_dataframe = '/content/tomato_dataframe_merlice_all_but06.csv'
#tomato_dataframe = '/content/tomato_dataframe_merlice_all_but07.csv'
#tomato_dataframe = '/content/tomato_dataframe_merlice_all_but08.csv'
#tomato_dataframe = '/content/tomato_dataframe_merlice_all_30cm.csv'
#tomato_dataframe = '/content/tomato_dataframe_merlice_all_60cm.csv'
#tomato_dataframe = '/content/tomato_dataframe_merlice_all_3050cm.csv'
#tomato_dataframe = '/content/tomato_dataframe_merlice_all_3060cm.csv'
#tomato_dataframe = '/content/tomato_dataframe_merlice_all_cleaned.csv'
#tomato_dataframe = '/content/tomato_dataframe.csv'

tomato_dataframe_test = '/content/tomato_dataframe_merlice_1x20d01.csv'
#tomato_dataframe_test = '/content/tomato_dataframe_merlice_1x20d04.csv'
#tomato_dataframe_test = '/content/tomato_dataframe_merlice_all_3060cm.csv'

#Training

In [ ]:
from pandas import read_csv
dataframe = read_csv(tomato_dataframe, sep=',', header=0)
print(dataframe)

In [ ]:
# Regression Example With Boston Dataset: Baseline
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
# load dataset
dataframe = read_csv(tomato_dataframe, sep=',', header=0)
#dataframe = read_csv(tomato_dataframe, sep=';', header=0)
dataset = dataframe.values
# split into input (X) and output (Y) variables
#X = dataframe[['distance','depth', 'pixels']].values
X = dataframe[['depth', 'pixels']].values
Y = dataframe[['weight']].values
# define base model
def baseline_model():
    # create model
    model = Sequential()
    #model.add(Dense(3, input_dim=3, kernel_initializer='normal', activation='relu'))
    model.add(Dense(2, input_dim=2, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    #Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X, Y, epochs=1000, verbose=0)
    return model
model = baseline_model()
print('Done Training')

**Normalized model**

In [ ]:
# Regression Example With Boston Dataset: Standardized
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

# load training dataset
dataframe = read_csv(tomato_dataframe, sep=',', header=0)
# dataframe = read_csv(tomato_dataframe, sep=';', header=0)
dataset = dataframe.values
# split into input (X) and output (Y) variables
#X = dataframe[['distance','depth', 'pixels']].values
#X = dataframe[['distance','pixels']].values
X = dataframe[['depth','pixels']].values
Y = dataframe[['weight']].values
# Parse numbers as floats
X = X.astype('float32')
Y = Y.astype('float32')

# create model
model = Sequential()
#model.add(Dense(3, input_dim=3, kernel_initializer='normal', activation='relu'))
model.add(Dense(2, input_dim=2, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(3, kernel_initializer='normal'))
model.add(Dense(1, kernel_initializer='normal'))

# root mean squared error (rmse) for regression (only for Keras tensors)
def rmse(y_true, y_pred):
    from keras import backend
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

# mean squared error (mse) for regression  (only for Keras tensors)
def mse(y_true, y_pred):
    from keras import backend
    return backend.mean(backend.square(y_pred - y_true), axis=-1)

# coefficient of determination (R^2) for regression  (only for Keras tensors)
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# Compile model
#model.compile(loss='mean_squared_error', optimizer='adam')
model.compile(optimizer="adam", loss=rmse, metrics=[r_square, rmse, mse])

# fit model
history = model.fit(X, Y, epochs=100, batch_size=32, verbose=1)

print('Done Training')

# Visualize history
# Plot history: Loss
plt.plot(history.history['loss'])
plt.title('Validation loss history')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
model.layers

**Evaluate dataset**

In [ ]:
# load dataset
dataframe_test = read_csv(tomato_dataframe_test, sep=',', header=0)
#dataframe = read_csv(tomato_dataframe, sep=';', header=0)
dataset = dataframe_test.values
# split into input (X) and output (Y) variables
#X = dataframe[['distance','depth', 'pixels']].values
X_test = dataframe[['depth', 'pixels']].values
Y_test = dataframe[['weight']].values
# Parse numbers as floats
X_test = X_test.astype('float32')
Y_test = Y_test.astype('float32')

# Generate generalization metrics
score = model.evaluate(X_test, Y_test)
print(score)

6/6 [==============================] - 0s 3ms/step - loss: 14.9930 - r_square: -611678272.0000 - rmse: 14.9930 - mse: 354.6921
[14.993024826049805, -611678272.0, 14.993024826049805, 354.6921081542969]


In [ ]:
history.history

In [ ]:
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=32, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -14612.80 (8660.01) MSE


In [ ]:
#Save the model
model.save('/content/model/')


INFO:tensorflow:Assets written to: /content/model/assets


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Masked images folder
%cd /content/model/

#Zip all files in this folder
!rm model.zip
!zip model.zip *

#Copy (.zip) files to (Google drive) destination
!cp -r /content/model/model.zip /content/drive/MyDrive/mask_rcnn/masks/

/content/model
rm: cannot remove 'model.zip': No such file or directory
  adding: assets/ (stored 0%)
  adding: keras_metadata.pb (deflated 84%)
  adding: saved_model.pb (deflated 86%)
  adding: variables/ (stored 0%)


#Prediction

In [ ]:
dataframe = read_csv(tomato_dataframe_test, sep=',', header=0)
#dataframe = read_csv(tomato_dataframe, sep=';', header=0)
dataset = dataframe.values
#Xpredict = dataframe[['distance','depth', 'pixels']].values
Xpredict = dataframe[['depth', 'pixels']].values
Yreal = dataframe[['weight']].values
totalerror =0
                     
# make a prediction
ypredict = model.predict(Xpredict)
# show the inputs and predicted outputs
for i in range(len(Xpredict)):
  error = abs(ypredict-Yreal[i])/Yreal[i]*100
  print("X=%s, Predicted=%s, Real=%s, Error=%s%%" % (Xpredict[i], ypredict[i], Yreal[i], error[i]))

print('Average error =' + str(sum(error)/len(error))+'%')